In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [ ]:
# https://www.kaggle.com/datasets/skillsmuggler/amazon-ratings
df=pd.read_csv('ratings_Beauty.csv')


In [ ]:
df=df.drop('Timestamp',axis=1)
LE=LabelEncoder()
df['UserId']=LE.fit_transform(df['UserId'])
df['ProductId']=LE.fit_transform(df['ProductId'])

In [ ]:
train,test=train_test_split(df,test_size=0.2)

In [ ]:
no_of_rated_products_per_user = df.groupby(by='UserId')['Rating'].count().sort_values(ascending=False)
no_of_rated_products_per_user.head(25)

UserId
234821    183
109821    175
69328     111
221860     68
167802     67
301635     60
304452     60
230890     53
87774      50
175167     47
296383     46
247789     46
298828     44
300657     43
120150     43
7926       42
182078     42
196732     37
109307     34
311235     34
169133     31
315013     31
269711     31
284465     31
62568      30
Name: Rating, dtype: int64

In [ ]:
nearest_neighbors=NearestNeighbors(metric='cosine',algorithm='brute',n_neighbors=5)

In [ ]:
nearest_neighbors.fit(train[['UserId','ProductId']])

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
import pandas as pd

def recommendation_system_function(userID, n_recommendation, nearest_neighbors):
    user_profile = df[df['UserId'] == userID].drop('Rating', axis=1)

    # Find similar users using nearest neighbors based on user profiles
    distances, indices = nearest_neighbors.kneighbors(user_profile)
    similar_users = indices.flatten()

    # Retrieve products rated by similar users
    user_recommendations = []
    for user in similar_users:
        products_rated_by_user = df[df['UserId'] == user]
        product_list = products_rated_by_user['ProductId'].unique()
        user_recommendations.extend(product_list)
    unique_recommendations = list(set(user_recommendations))

    # Calculate the number of ratings for each recommended product
    product_ratings_count = df[df['ProductId'].isin(unique_recommendations)]['ProductId'].value_counts()

    # Sort the products based on ratings count in descending order
    sorted_recommendations = product_ratings_count.sort_values(ascending=False)

    # Select the top-rated recommendations
    top_recommendations = sorted_recommendations.index[:n_recommendation]

    # Convert recommended product IDs back to original form
    recommended_product_codes = LE.inverse_transform(top_recommendations)

    # Create a DataFrame to store the recommended products and their data
    recommendation_table = pd.DataFrame({'Product ID': top_recommendations, 'Product Code': recommended_product_codes})
    recommendation_table['Rating Count'] = recommendation_table['Product ID'].map(product_ratings_count)
    recommendation_table.set_index('Product ID', inplace=True)

    # Print the recommendation table
    print("Recommended Products:")
    print(recommendation_table)

    return recommended_product_codes

In [ ]:
print(df[df['ProductId']=='B003BMGA6C']['UserId'])

Series([], Name: UserId, dtype: int64)


In [ ]:
user_id = 105  # Replace with the desired user ID
num_recommendations = 10  # Replace with the desired number of recommendations

recommended_products = recommendation_system_function(user_id, num_recommendations, nearest_neighbors)

Recommended Products:
           Product Code  Rating Count
Product ID                           
23255        B000KFX10G           406
3641         B0001EKWPI           195
5173         B00028MLKC            48
13907        B000C213WA            41
19327        B000GLB37C             4
